In [38]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os

with random coordinates

In [39]:
# reading the file

arquivo = "data/us-post-offices-random-coords.csv"

us_post_offices_raw = pd.read_csv(arquivo, delimiter=",")

us_post_offices_raw

C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipykernel_13844\3399243105.py:5: DtypeWarning: Columns (5,6,11) have mixed types. Specify dtype option on import or set low_memory=False.
  us_post_offices_raw = pd.read_csv(arquivo, delimiter=",")


,Name,AltName,OrigName,State,County1,County2,County3,OrigCounty,Established,Discontinued,...,GNIS.Feature.Class,GNIS.OrigName,GNIS.OrigCounty,GNIS.Latitude,GNIS.Longitude,GNIS.ELEV_IN_M,GNIS.Dist,Latitude,Longitude,RandomCoordsFlag
0,A AND M COLLEGE,NaN,A AND M COLLEGE,MS,OKTIBBEHA,NaN,NaN,Oktibbeha,1922.0,1932.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.508712,-88.703363,True
1,A. B. C.,NaN,A. B. C.,TN,SUMNER,NaN,NaN,Sumner,1880.0,1914.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.419931,-86.563870,True
2,AARON,NaN,AARON,MO,BATES,NaN,NaN,Bates,1895.0,1933.0,...,Post Office,Aaron Post Office (historical),Bates,38.422222,-94.154167,268.0,1.0,38.422222,-94.154167,False
3,AARON,NaN,AARON,TN,BENTON,NaN,NaN,Benton,1889.0,1890.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.205923,-88.094459,True
4,AARON,NaN,AARON,TN,BRADLEY,NaN,NaN,Bradley,1860.0,1860.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.143874,-84.931121,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166135,TRUE,NaN,TRUE,OK,HARMON,NaN,NaN,Harmon,1901.0,1905.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.738756,-99.782970,True
166136,TRUE,NaN,TRUE,TX,YOUNG,NaN,NaN,Young,1894.0,1912.0,...,Populated Place,True,Young,33.279830,-98.728670,359.0,1.0,33.279830,-98.728670,False
166137,SIERRA BLANCA,NaN,SIERRA BLANCA,TX,HUDSPETH,NaN,NaN,Hudspeth,1882.0,NaN,...,Post Office,Sierra Blanca Post Office,Hudspeth,31.175930,-105.360000,1380.0,1.0,31.175930,-105.360000,False
166138,GRAHAM'S MILLS,GRAHAMS MILLS,GRAHAM'S MILLS,TX,SHELBY,NaN,NaN,Shelby,1854.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.962916,-94.393446,True


In [ ]:
# Filtering out the Post Office with ID 127138 (Bexar, Texas), as it appears to be an outlier.
# The recorded establishment year is 1779, which is unusual since Texas was not part of the US at that time.
# This data point is likely incorrect and does not align with the historical population expansion
# and establishment of Post Offices in the US during that period.

us_post_offices_raw = us_post_offices_raw[us_post_offices_raw['ID'] != 127138]

In [40]:
us_post_offices_raw = us_post_offices_raw.dropna(subset=['Established'])

In [41]:
# getting only the columns that are needed

us_post_offices_raw_cleaned = us_post_offices_raw.loc[:,['Name','GNIS.State', 'State', 'County1', 'GNIS.OrigCounty','Established','Discontinued', 'Latitude','Longitude']]

us_post_offices_raw_cleaned

,Name,GNIS.State,State,County1,GNIS.OrigCounty,Established,Discontinued,Latitude,Longitude
0,A AND M COLLEGE,NaN,MS,OKTIBBEHA,NaN,1922.0,1932.0,33.508712,-88.703363
1,A. B. C.,NaN,TN,SUMNER,NaN,1880.0,1914.0,36.419931,-86.563870
2,AARON,MO,MO,BATES,Bates,1895.0,1933.0,38.422222,-94.154167
3,AARON,NaN,TN,BENTON,NaN,1889.0,1890.0,36.205923,-88.094459
4,AARON,NaN,TN,BRADLEY,NaN,1860.0,1860.0,35.143874,-84.931121
...,...,...,...,...,...,...,...,...,...
166135,TRUE,NaN,OK,HARMON,NaN,1901.0,1905.0,34.738756,-99.782970
166136,TRUE,TX,TX,YOUNG,Young,1894.0,1912.0,33.279830,-98.728670
166137,SIERRA BLANCA,TX,TX,HUDSPETH,Hudspeth,1882.0,NaN,31.175930,-105.360000
166138,GRAHAM'S MILLS,NaN,TX,SHELBY,NaN,1854.0,NaN,31.962916,-94.393446


In [42]:
# adjusting null values by combining two complementary columns

us_post_offices_raw_cleaned['GNIS.State'] = us_post_offices_raw_cleaned['GNIS.State'].fillna(us_post_offices_raw_cleaned['State'])
us_post_offices_raw_cleaned['GNIS.OrigCounty'] = us_post_offices_raw_cleaned['GNIS.OrigCounty'].fillna(us_post_offices_raw_cleaned['County1'])

us_post_offices_raw_cleaned

,Name,GNIS.State,State,County1,GNIS.OrigCounty,Established,Discontinued,Latitude,Longitude
0,A AND M COLLEGE,MS,MS,OKTIBBEHA,OKTIBBEHA,1922.0,1932.0,33.508712,-88.703363
1,A. B. C.,TN,TN,SUMNER,SUMNER,1880.0,1914.0,36.419931,-86.563870
2,AARON,MO,MO,BATES,Bates,1895.0,1933.0,38.422222,-94.154167
3,AARON,TN,TN,BENTON,BENTON,1889.0,1890.0,36.205923,-88.094459
4,AARON,TN,TN,BRADLEY,BRADLEY,1860.0,1860.0,35.143874,-84.931121
...,...,...,...,...,...,...,...,...,...
166135,TRUE,OK,OK,HARMON,HARMON,1901.0,1905.0,34.738756,-99.782970
166136,TRUE,TX,TX,YOUNG,Young,1894.0,1912.0,33.279830,-98.728670
166137,SIERRA BLANCA,TX,TX,HUDSPETH,Hudspeth,1882.0,NaN,31.175930,-105.360000
166138,GRAHAM'S MILLS,TX,TX,SHELBY,SHELBY,1854.0,NaN,31.962916,-94.393446


In [43]:
us_post_offices_raw_cleaned = us_post_offices_raw_cleaned[['Name','GNIS.State','Established','Discontinued','Latitude','Longitude']]

us_post_offices_raw_cleaned

,Name,GNIS.State,Established,Discontinued,Latitude,Longitude
0,A AND M COLLEGE,MS,1922.0,1932.0,33.508712,-88.703363
1,A. B. C.,TN,1880.0,1914.0,36.419931,-86.563870
2,AARON,MO,1895.0,1933.0,38.422222,-94.154167
3,AARON,TN,1889.0,1890.0,36.205923,-88.094459
4,AARON,TN,1860.0,1860.0,35.143874,-84.931121
...,...,...,...,...,...,...
166135,TRUE,OK,1901.0,1905.0,34.738756,-99.782970
166136,TRUE,TX,1894.0,1912.0,33.279830,-98.728670
166137,SIERRA BLANCA,TX,1882.0,NaN,31.175930,-105.360000
166138,GRAHAM'S MILLS,TX,1854.0,NaN,31.962916,-94.393446


In [44]:
# filter the DataFrame to include only rows where:
# 1. the 'Established' year is between 1600 and 2000 (inclusive)
# 2. the 'Discontinued' year is either greater than or equal to 1600, 
#    or the value is missing (NaN).
us_post_offices_raw_cleaned = us_post_offices_raw_cleaned[
    # first condition: 'Established' year is between 1600 and 2000
    ((us_post_offices_raw_cleaned['Established'] >= 1600) & 
     (us_post_offices_raw_cleaned['Established'] <= 2000)) & 
    
    # second condition: 'Discontinued' year is either greater than or equal to 1600 
    # or the 'Discontinued' column is NaN (missing values).
    ((us_post_offices_raw_cleaned['Discontinued'] >= 1600) | 
     (pd.isnull(us_post_offices_raw_cleaned['Discontinued'])))
]


us_post_offices_raw_cleaned

,Name,GNIS.State,Established,Discontinued,Latitude,Longitude
0,A AND M COLLEGE,MS,1922.0,1932.0,33.508712,-88.703363
1,A. B. C.,TN,1880.0,1914.0,36.419931,-86.563870
2,AARON,MO,1895.0,1933.0,38.422222,-94.154167
3,AARON,TN,1889.0,1890.0,36.205923,-88.094459
4,AARON,TN,1860.0,1860.0,35.143874,-84.931121
...,...,...,...,...,...,...
166135,TRUE,OK,1901.0,1905.0,34.738756,-99.782970
166136,TRUE,TX,1894.0,1912.0,33.279830,-98.728670
166137,SIERRA BLANCA,TX,1882.0,NaN,31.175930,-105.360000
166138,GRAHAM'S MILLS,TX,1854.0,NaN,31.962916,-94.393446


In [45]:
# replace NaN values in the 'Discontinued' column with 0
us_post_offices_raw_cleaned['Discontinued'] = us_post_offices_raw_cleaned['Discontinued'].fillna(0)

# convert the 'Established' and 'Discontinued' columns to integers
us_post_offices_raw_cleaned['Established'] = us_post_offices_raw_cleaned['Established'].astype(int)
us_post_offices_raw_cleaned['Discontinued'] = us_post_offices_raw_cleaned['Discontinued'].astype(int)

us_post_offices_raw_cleaned

C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipykernel_13844\553237924.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_post_offices_raw_cleaned['Discontinued'] = us_post_offices_raw_cleaned['Discontinued'].fillna(0)
C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipykernel_13844\553237924.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_post_offices_raw_cleaned['Established'] = us_post_offices_raw_cleaned['Established'].astype(int)
C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipykernel

,Name,GNIS.State,Established,Discontinued,Latitude,Longitude
0,A AND M COLLEGE,MS,1922,1932,33.508712,-88.703363
1,A. B. C.,TN,1880,1914,36.419931,-86.563870
2,AARON,MO,1895,1933,38.422222,-94.154167
3,AARON,TN,1889,1890,36.205923,-88.094459
4,AARON,TN,1860,1860,35.143874,-84.931121
...,...,...,...,...,...,...
166135,TRUE,OK,1901,1905,34.738756,-99.782970
166136,TRUE,TX,1894,1912,33.279830,-98.728670
166137,SIERRA BLANCA,TX,1882,0,31.175930,-105.360000
166138,GRAHAM'S MILLS,TX,1854,0,31.962916,-94.393446


In [46]:
years = list(range(1790, 2010, 10))

In [47]:
# initialize an empty list to store the mean centers for each year
mean_centers = []

# loop through each year in the list of years
for year in years:
    # filter the post offices that were established before or in the current year
    # and are either still active (not discontinued or discontinued after the current year) 
    active_post_offices = us_post_offices_raw_cleaned[
        (us_post_offices_raw_cleaned['Established'] <= year) & 
        ((us_post_offices_raw_cleaned['Discontinued'] > year) | 
         (us_post_offices_raw_cleaned['Discontinued'] == 0))
    ]
    
    # if there are active post offices for the current year, calculate their mean location
    if not active_post_offices.empty:
        # calculate the mean latitude
        mean_lat = active_post_offices['Latitude'].mean()

        # convert latitude to radians for calculating the mean longitude
        active_post_offices['Lat_rad'] = np.radians(active_post_offices['Latitude'])

        # calculate the weighted mean longitude using the latitude values
        mean_lon = (
            active_post_offices['Longitude'] * np.cos(active_post_offices['Lat_rad'])
        ).sum() / np.cos(active_post_offices['Lat_rad']).sum()

        # append the results for this year (mean latitude and longitude)
        mean_centers.append({'Year': year, 'Mean_Latitude': mean_lat, 'Mean_Longitude': mean_lon})
        
    # if there are no active post offices for the current year, append None for mean coordinates
    else:
        mean_centers.append({'Year': year, 'Mean_Latitude': None, 'Mean_Longitude': None})


C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipykernel_13844\1432278187.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_post_offices['Lat_rad'] = np.radians(active_post_offices['Latitude'])
C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipykernel_13844\1432278187.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_post_offices['Lat_rad'] = np.radians(active_post_offices['Latitude'])
C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipykernel_13844\1432278187.py:20: SettingWithCopyWa

In [48]:
result_df = pd.DataFrame(mean_centers)

result_df

,Year,Mean_Latitude,Mean_Longitude
0,1790,39.885246,-75.165341
1,1800,39.870022,-75.984666
2,1810,39.895449,-76.899163
3,1820,39.776280,-78.056850
4,1830,39.600737,-78.964689
5,1840,39.347771,-80.594858
6,1850,39.185561,-81.799291
7,1860,39.136415,-83.879322
8,1870,39.506581,-84.747517
9,1880,39.200004,-86.524739


In [49]:
result_df.to_excel("tests_project_us_post_offices_states/mean_center_post_offices_random_coordinates.xlsx")

without random coordinates

In [50]:
#reading the file

arquivo_ = "C:/Users/AnaLíviadosSantosMaz/Downloads/us-post-offices.csv"

us_post_offices_raw_ = pd.read_csv(arquivo_)

us_post_offices_raw_

C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipykernel_13844\2394745168.py:5: DtypeWarning: Columns (5,6,11) have mixed types. Specify dtype option on import or set low_memory=False.
  us_post_offices_raw_ = pd.read_csv(arquivo_)


,Name,AltName,OrigName,State,County1,County2,County3,OrigCounty,Established,Discontinued,...,GNIS.FEATURE_ID,GNIS.Feature.Class,GNIS.OrigName,GNIS.OrigCounty,GNIS.Latitude,GNIS.Longitude,GNIS.ELEV_IN_M,GNIS.Dist,Latitude,Longitude
0,A AND M COLLEGE,NaN,A AND M COLLEGE,MS,OKTIBBEHA,NaN,NaN,Oktibbeha,1922.0,1932.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A. B. C.,NaN,A. B. C.,TN,SUMNER,NaN,NaN,Sumner,1880.0,1914.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AARON,NaN,AARON,MO,BATES,NaN,NaN,Bates,1895.0,1933.0,...,2091668.0,Post Office,Aaron Post Office (historical),Bates,38.422222,-94.154167,268.0,1.0,38.422222,-94.154167
3,AARON,NaN,AARON,TN,BENTON,NaN,NaN,Benton,1889.0,1890.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AARON,NaN,AARON,TN,BRADLEY,NaN,NaN,Bradley,1860.0,1860.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166135,TRUE,NaN,TRUE,OK,HARMON,NaN,NaN,Harmon,1901.0,1905.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166136,TRUE,NaN,TRUE,TX,YOUNG,NaN,NaN,Young,1894.0,1912.0,...,1380679.0,Populated Place,True,Young,33.279830,-98.728670,359.0,1.0,33.279830,-98.728670
166137,SIERRA BLANCA,NaN,SIERRA BLANCA,TX,HUDSPETH,NaN,NaN,Hudspeth,1882.0,NaN,...,2732001.0,Post Office,Sierra Blanca Post Office,Hudspeth,31.175930,-105.360000,1380.0,1.0,31.175930,-105.360000
166138,GRAHAM'S MILLS,GRAHAMS MILLS,GRAHAM'S MILLS,TX,SHELBY,NaN,NaN,Shelby,1854.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
us_post_offices_raw_ = us_post_offices_raw_.dropna(subset=['Latitude', 'Longitude'])

us_post_offices_raw_

,Name,AltName,OrigName,State,County1,County2,County3,OrigCounty,Established,Discontinued,...,GNIS.FEATURE_ID,GNIS.Feature.Class,GNIS.OrigName,GNIS.OrigCounty,GNIS.Latitude,GNIS.Longitude,GNIS.ELEV_IN_M,GNIS.Dist,Latitude,Longitude
2,AARON,NaN,AARON,MO,BATES,NaN,NaN,Bates,1895.0,1933.0,...,2091668.0,Post Office,Aaron Post Office (historical),Bates,38.422222,-94.154167,268.0,1.0,38.422222,-94.154167
5,AARON,NaN,AARON,GA,BULLOCH,NaN,NaN,Bulloch,1909.0,1920.0,...,326146.0,Populated Place,Aaron,Bulloch,32.568220,-81.992340,81.0,1.0,32.568220,-81.992340
7,AARON,NaN,AARON,SC,ANDERSON,NaN,NaN,Anderson,1892.0,1899.0,...,1253367.0,Post Office,Aaron Post Office (historical),Anderson,34.577053,-82.623466,265.0,1.0,34.577053,-82.623466
8,AARONSBURG,AARONSBURGH,AARONSBURG(H),PA,CENTRE,NaN,NaN,Centre,1792.0,NaN,...,2779639.0,Post Office,Aaronsburg Post Office,Centre,40.898967,-77.452199,348.0,1.0,40.898967,-77.452199
11,ABADYL,NaN,ABADYL,MO,CHRISTIAN,NaN,NaN,Christian,1895.0,1919.0,...,740613.0,Populated Place,Abadyl,Christian,37.038386,-92.947119,402.0,1.0,37.038386,-92.947119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166129,ZURICH,NaN,ZURICH,MT,BLAINE,NaN,NaN,Blaine,1907.0,NaN,...,1722458.0,Post Office,Zurich Post Office,Blaine,48.584330,-109.031700,726.0,1.0,48.584330,-109.031700
166131,ZYBA,NaN,ZYBA,KS,SUMNER,NaN,NaN,Sumner,1887.0,1906.0,...,484508.0,Populated Place,Zyba,Sumner,37.434740,-97.389210,375.0,1.0,37.434740,-97.389210
166133,ZYBACH,NaN,ZYBACH,TX,WHEELER,NaN,NaN,Wheeler,1910.0,1931.0,...,1372110.0,Cemetery,Zybach Cemetery,Wheeler,35.619210,-100.190700,817.0,1.0,35.619210,-100.190700
166136,TRUE,NaN,TRUE,TX,YOUNG,NaN,NaN,Young,1894.0,1912.0,...,1380679.0,Populated Place,True,Young,33.279830,-98.728670,359.0,1.0,33.279830,-98.728670


In [52]:
us_post_offices_raw_ = us_post_offices_raw.dropna(subset=['Established'])

In [53]:
# getting only the columns that are needed

us_post_offices_raw_cleaned_ = us_post_offices_raw_.loc[:,['Name','GNIS.State', 'State', 'County1', 'GNIS.OrigCounty','Established','Discontinued','Latitude','Longitude']]

us_post_offices_raw_cleaned_

,Name,GNIS.State,State,County1,GNIS.OrigCounty,Established,Discontinued,Latitude,Longitude
0,A AND M COLLEGE,NaN,MS,OKTIBBEHA,NaN,1922.0,1932.0,33.508712,-88.703363
1,A. B. C.,NaN,TN,SUMNER,NaN,1880.0,1914.0,36.419931,-86.563870
2,AARON,MO,MO,BATES,Bates,1895.0,1933.0,38.422222,-94.154167
3,AARON,NaN,TN,BENTON,NaN,1889.0,1890.0,36.205923,-88.094459
4,AARON,NaN,TN,BRADLEY,NaN,1860.0,1860.0,35.143874,-84.931121
...,...,...,...,...,...,...,...,...,...
166135,TRUE,NaN,OK,HARMON,NaN,1901.0,1905.0,34.738756,-99.782970
166136,TRUE,TX,TX,YOUNG,Young,1894.0,1912.0,33.279830,-98.728670
166137,SIERRA BLANCA,TX,TX,HUDSPETH,Hudspeth,1882.0,NaN,31.175930,-105.360000
166138,GRAHAM'S MILLS,NaN,TX,SHELBY,NaN,1854.0,NaN,31.962916,-94.393446


In [54]:
# adjusting null values by combining two complementary columns

us_post_offices_raw_cleaned_['GNIS.State'] = us_post_offices_raw_cleaned_['GNIS.State'].fillna(us_post_offices_raw_cleaned_['State'])
us_post_offices_raw_cleaned_['GNIS.OrigCounty'] = us_post_offices_raw_cleaned_['GNIS.OrigCounty'].fillna(us_post_offices_raw_cleaned_['County1'])

us_post_offices_raw_cleaned_

,Name,GNIS.State,State,County1,GNIS.OrigCounty,Established,Discontinued,Latitude,Longitude
0,A AND M COLLEGE,MS,MS,OKTIBBEHA,OKTIBBEHA,1922.0,1932.0,33.508712,-88.703363
1,A. B. C.,TN,TN,SUMNER,SUMNER,1880.0,1914.0,36.419931,-86.563870
2,AARON,MO,MO,BATES,Bates,1895.0,1933.0,38.422222,-94.154167
3,AARON,TN,TN,BENTON,BENTON,1889.0,1890.0,36.205923,-88.094459
4,AARON,TN,TN,BRADLEY,BRADLEY,1860.0,1860.0,35.143874,-84.931121
...,...,...,...,...,...,...,...,...,...
166135,TRUE,OK,OK,HARMON,HARMON,1901.0,1905.0,34.738756,-99.782970
166136,TRUE,TX,TX,YOUNG,Young,1894.0,1912.0,33.279830,-98.728670
166137,SIERRA BLANCA,TX,TX,HUDSPETH,Hudspeth,1882.0,NaN,31.175930,-105.360000
166138,GRAHAM'S MILLS,TX,TX,SHELBY,SHELBY,1854.0,NaN,31.962916,-94.393446


In [55]:
us_post_offices_raw_cleaned_ = us_post_offices_raw_cleaned_[['Name','GNIS.State','Established','Discontinued','Latitude','Longitude']]

us_post_offices_raw_cleaned_

,Name,GNIS.State,Established,Discontinued,Latitude,Longitude
0,A AND M COLLEGE,MS,1922.0,1932.0,33.508712,-88.703363
1,A. B. C.,TN,1880.0,1914.0,36.419931,-86.563870
2,AARON,MO,1895.0,1933.0,38.422222,-94.154167
3,AARON,TN,1889.0,1890.0,36.205923,-88.094459
4,AARON,TN,1860.0,1860.0,35.143874,-84.931121
...,...,...,...,...,...,...
166135,TRUE,OK,1901.0,1905.0,34.738756,-99.782970
166136,TRUE,TX,1894.0,1912.0,33.279830,-98.728670
166137,SIERRA BLANCA,TX,1882.0,NaN,31.175930,-105.360000
166138,GRAHAM'S MILLS,TX,1854.0,NaN,31.962916,-94.393446


In [56]:
us_post_offices_raw_cleaned_ = us_post_offices_raw_cleaned_[
    ((us_post_offices_raw_cleaned_['Established'] >= 1600) & (us_post_offices_raw_cleaned_['Established'] <= 2000)) &
    ((us_post_offices_raw_cleaned_['Discontinued'] >= 1600) |(pd.isnull(us_post_offices_raw_cleaned_['Discontinued'])))
]

us_post_offices_raw_cleaned_

,Name,GNIS.State,Established,Discontinued,Latitude,Longitude
0,A AND M COLLEGE,MS,1922.0,1932.0,33.508712,-88.703363
1,A. B. C.,TN,1880.0,1914.0,36.419931,-86.563870
2,AARON,MO,1895.0,1933.0,38.422222,-94.154167
3,AARON,TN,1889.0,1890.0,36.205923,-88.094459
4,AARON,TN,1860.0,1860.0,35.143874,-84.931121
...,...,...,...,...,...,...
166135,TRUE,OK,1901.0,1905.0,34.738756,-99.782970
166136,TRUE,TX,1894.0,1912.0,33.279830,-98.728670
166137,SIERRA BLANCA,TX,1882.0,NaN,31.175930,-105.360000
166138,GRAHAM'S MILLS,TX,1854.0,NaN,31.962916,-94.393446


In [57]:
us_post_offices_raw_cleaned_['Discontinued'] = us_post_offices_raw_cleaned_['Discontinued'].fillna(0)

us_post_offices_raw_cleaned_['Established'] = us_post_offices_raw_cleaned_['Established'].astype(int)
us_post_offices_raw_cleaned_['Discontinued'] = us_post_offices_raw_cleaned_['Discontinued'].astype(int)

us_post_offices_raw_cleaned_

C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipykernel_13844\737315665.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_post_offices_raw_cleaned_['Discontinued'] = us_post_offices_raw_cleaned_['Discontinued'].fillna(0)
C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipykernel_13844\737315665.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_post_offices_raw_cleaned_['Established'] = us_post_offices_raw_cleaned_['Established'].astype(int)
C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipyke

,Name,GNIS.State,Established,Discontinued,Latitude,Longitude
0,A AND M COLLEGE,MS,1922,1932,33.508712,-88.703363
1,A. B. C.,TN,1880,1914,36.419931,-86.563870
2,AARON,MO,1895,1933,38.422222,-94.154167
3,AARON,TN,1889,1890,36.205923,-88.094459
4,AARON,TN,1860,1860,35.143874,-84.931121
...,...,...,...,...,...,...
166135,TRUE,OK,1901,1905,34.738756,-99.782970
166136,TRUE,TX,1894,1912,33.279830,-98.728670
166137,SIERRA BLANCA,TX,1882,0,31.175930,-105.360000
166138,GRAHAM'S MILLS,TX,1854,0,31.962916,-94.393446


In [58]:
mean_centers_ = []


for year in years:
    active_post_offices_ = us_post_offices_raw_cleaned_[
        (us_post_offices_raw_cleaned_['Established'] <= year) & 
        ((us_post_offices_raw_cleaned_['Discontinued'] > year) | 
         (us_post_offices_raw_cleaned_['Discontinued'] == 0))
    ]
    
    if not active_post_offices_.empty:
        #Mean latitude
        mean_lat_ = active_post_offices_['Latitude'].mean()

        #Mean longitude
        active_post_offices_['Lat_rad'] = np.radians(active_post_offices_['Latitude'])
        mean_lon_ = (
            active_post_offices_['Longitude'] * np.cos(active_post_offices_['Lat_rad'])
        ).sum() / np.cos(active_post_offices_['Lat_rad']).sum()

        mean_centers_.append({'Year': year, 'Mean_Latitude': mean_lat_, 'Mean_Longitude': mean_lon_})
        
    else:
        mean_centers_.append({'Year': year, 'Mean_Latitude': None, 'Mean_Longitude': None})


C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipykernel_13844\333638390.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_post_offices_['Lat_rad'] = np.radians(active_post_offices_['Latitude'])
C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipykernel_13844\333638390.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_post_offices_['Lat_rad'] = np.radians(active_post_offices_['Latitude'])
C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipykernel_13844\333638390.py:16: SettingWithCopyW

In [59]:
result_df_ = pd.DataFrame(mean_centers_)

result_df_

,Year,Mean_Latitude,Mean_Longitude
0,1790,39.885246,-75.165341
1,1800,39.870022,-75.984666
2,1810,39.895449,-76.899163
3,1820,39.776280,-78.056850
4,1830,39.600737,-78.964689
5,1840,39.347771,-80.594858
6,1850,39.185561,-81.799291
7,1860,39.136415,-83.879322
8,1870,39.506581,-84.747517
9,1880,39.200004,-86.524739


In [60]:
result_df_.to_excel("tests_project_us_post_offices_states/mean_center_post_offices.xlsx")